In [2]:
from ultralytics import YOLO

# Load a pre-trained model (v8n = nano, fast; v8s = small, more accurate)
model = YOLO("yolov8n.pt")

# Train on your custom dataset
model.train(
    data="D:\Project_train\data.yaml",  # path to your data.yaml
    epochs=100,                     # number of epochs (increase if needed)
    imgsz=640,                       # image size (adjust if GPU allows)
    batch=16,                        # batch size
    name="vehicle_detection_custom", # run name
    project="project_train"          # folder where outputs are saved
)


Ultralytics 8.3.218  Python-3.11.2 torch-2.9.0+cpu CPU (AMD Ryzen 7 4800H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Project_train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=vehicle_detection_custom2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000214DE9AD510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
    

In [3]:
# Evaluate on validation set
metrics = model.val()
print(metrics)  # prints mAP, precision, recall, etc.


Ultralytics 8.3.218  Python-3.11.2 torch-2.9.0+cpu CPU (AMD Ryzen 7 4800H with Radeon Graphics)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.0 ms, read: 220.00.0 MB/s, size: 104.1 KB)
val: Scanning D:\Project_train\valid\labels.cache... 1 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1/1 657.8it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.3it/s 0.2s
                   all          1         54       0.78      0.199      0.365      0.189
                  Bike          1          7          1          0          0          0
                    HV          1          2          0          0      0.181      0.119
                  auto          1          3          1          0          0          0
                   car          1         22          1      0.542      0.864      0.374
                 truck          1         20     

In [ ]:
import re
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from math import radians, cos, sin, sqrt, atan2
import os
import chardet

# === CONFIGURATION ===
VIDEO_PATH = "video1.MP4"
SRT_PATH = VIDEO_PATH.replace(".MP4", ".SRT")
OUTPUT_PATH = "output_speed_overlay.mp4"
FRAME_INTERVAL_SEC = 4  # process every 4 seconds
SENSOR_WIDTH_MM = 13.2  # camera sensor width
IMAGE_WIDTH_PX = 5472   # original width of the video
MODEL_PATH = "vehicle_detection_custom2/weights/best.pt"

# === HELPER FUNCTIONS ===
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    d_lat = radians(lat2 - lat1)
    d_lon = radians(lon2 - lon1)
    a = sin(d_lat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(d_lon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

def parse_srt(srt_path):
    # Detect encoding automatically
    with open(srt_path, "rb") as f:
        raw = f.read()
    enc = chardet.detect(raw)["encoding"] or "utf-8"
    text = raw.decode(enc, errors="ignore")

    frame_data = []
    blocks = text.strip().split("\n\n")
    for block in blocks:
        diff_match = re.search(r"DiffTime:\s*(\d+)ms", block)
        alt_match = re.search(r"rel_alt:\s*([\d.]+)", block)
        focal_match = re.search(r"focal_len:\s*([\d.]+)", block)
        lat_match = re.search(r"latitude:\s*([\d.]+)", block)
        lon_match = re.search(r"longitude:\s*([\d.]+)", block)

        diff_time = float(diff_match.group(1))/1000 if diff_match else 0.04
        rel_alt = float(alt_match.group(1)) if alt_match else 226.5
        focal_len = float(focal_match.group(1)) if focal_match else 24.0
        latitude = float(lat_match.group(1)) if lat_match else 0.0
        longitude = float(lon_match.group(1)) if lon_match else 0.0

        frame_data.append({
            "diff_time": diff_time,
            "altitude": rel_alt,
            "focal_len": focal_len,
            "lat": latitude,
            "lon": longitude
        })
    return frame_data

def compute_gsd(sensor_width_mm, focal_len_mm, altitude_m, img_width_px):
    """Ground Sampling Distance (meters per pixel)"""
    return (sensor_width_mm * altitude_m) / (focal_len_mm * img_width_px)

# === MAIN PROCESS ===
def main():
    # Load telemetry
    frame_meta = parse_srt(SRT_PATH)
    print(f"✅ Loaded {len(frame_meta)} frames metadata")

    # Open video
    cap = cv2.VideoCapture(VIDEO_PATH)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Prepare output
    out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    # Load YOLO model
    model = YOLO(MODEL_PATH)

    # Tracking structures
    last_positions = {}
    speeds = defaultdict(list)
    last_gps = None

    frame_idx = 0
    processed_frame_idx = 0
    interval_frames = int(FRAME_INTERVAL_SEC * fps)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Only process every Nth frame to save memory
        if frame_idx % interval_frames != 0:
            out.write(frame)
            frame_idx += 1
            continue

        # Metadata
        if frame_idx < len(frame_meta):
            meta = frame_meta[frame_idx]
            dt = meta["diff_time"]
            alt = meta["altitude"]
            focal = meta["focal_len"]
            lat, lon = meta["lat"], meta["lon"]
        else:
            dt, alt, focal, lat, lon = 0.04, 226.5, 24.0, 0.0, 0.0

        gsd = compute_gsd(SENSOR_WIDTH_MM, focal, alt, IMAGE_WIDTH_PX)

        # Drone motion compensation
        drone_shift_m = 0.0
        if last_gps is not None and lat != 0.0 and lon != 0.0:
            drone_shift_m = haversine(last_gps[0], last_gps[1], lat, lon)
        last_gps = (lat, lon)

        # YOLO detection + tracking
        results = model.track(frame, persist=True, classes=[2,3,5,7])  # car, bike, bus, truck
        if results[0].boxes.id is None:
            out.write(frame)
            frame_idx += 1
            continue

        ids = results[0].boxes.id.cpu().numpy().astype(int)
        boxes = results[0].boxes.xyxy.cpu().numpy()

        for box, obj_id in zip(boxes, ids):
            x1, y1, x2, y2 = box
            cx, cy = int((x1+x2)/2), int((y1+y2)/2)

            if obj_id in last_positions:
                px, py = last_positions[obj_id]
                pixel_dist = np.sqrt((cx-px)**2 + (cy-py)**2)
                real_dist = max(pixel_dist*gsd - drone_shift_m, 0)
                speed_mps = real_dist / dt
                speed_kph = speed_mps * 3.6
                speeds[obj_id].append(speed_kph)
            else:
                speed_kph = 0.0
                speeds[obj_id].append(speed_kph)

            last_positions[obj_id] = (cx, cy)
            avg_speed = np.mean(speeds[obj_id])

            # Overlay
            label = f"ID:{obj_id} | {speed_kph:.1f} km/h | Avg:{avg_speed:.1f}"
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)), (0,255,0), 2)
            cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        out.write(frame)
        frame_idx += 1
        processed_frame_idx += 1

    cap.release()
    out.release()
    print(f"✅ Done! Output saved at {OUTPUT_PATH}")

if __name__ == "__main__":
    main()


✅ Loaded 10030 frames metadata

0: 384x640 3 cars, 69.5ms
Speed: 19.2ms preprocess, 69.5ms inference, 13.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 86.8ms
Speed: 3.0ms preprocess, 86.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 86.4ms
Speed: 2.7ms preprocess, 86.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 91.3ms
Speed: 3.2ms preprocess, 91.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 119.5ms
Speed: 5.1ms preprocess, 119.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 97.5ms
Speed: 3.2ms preprocess, 97.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


In [1]:
from ultralytics import YOLO

# Load your custom model
model = YOLO("vehicle_detection_custom2/weights/best.pt")


In [5]:
# Run inference on a video file
results = model.predict(
    source="video1.MP4",  # input video
    conf=0.01,                        # confidence threshold
    save=True,                        # saves output video automatically
    show=True                         # displays it in a window (optional)
)



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/10030) D:\Project_train\video1.MP4: 384x640 20 HVs, 17 cars, 7 trucks, 125.5ms
video 1/1 (frame 2/10030) D:\Project_train\video1.MP4: 384x640 19 HVs, 15 cars, 8 trucks, 87.0ms
video 1/1 (frame 3/10030) D:\Project_train\video1.MP4: 384x640 20 HVs, 18 cars, 6 trucks, 92.8ms
video 1/1 (frame 4/10030) D:\Project_train\video1.MP4: 384x640 20 HVs, 15 cars, 8 trucks, 107.9ms
video 1/1 (frame 5/10030) D:\Project_train\video1.MP4: 384x640 21 HVs

KeyboardInterrupt: 

In [6]:
import re
from datetime import datetime, timedelta

def parse_srt(srt_path):
    """
    Extracts time and GPS data from .srt metadata file.
    Returns a list of (timestamp_in_seconds, latitude, longitude, altitude)
    """
    data = []
    with open(srt_path, 'r') as f:
        content = f.read()

    pattern = r"(\d+)\n(\d{2}:\d{2}:\d{2},\d{3}) --> .*?\nGPS: ([\d\.\-]+), ([\d\.\-]+), ([\d\.\-]+)"
    matches = re.findall(pattern, content)

    for _, time_str, lat, lon, alt in matches:
        # Convert to seconds
        t = datetime.strptime(time_str, "%H:%M:%S,%f")
        seconds = t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6
        data.append((seconds, float(lat), float(lon), float(alt)))

    return data


In [7]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius (meters)
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)

    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlambda/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))


In [8]:
def compute_speed_from_gps(gps_data):
    """
    Returns list of (time, speed_mps) using consecutive GPS points
    """
    speeds = []
    for i in range(1, len(gps_data)):
        t1, lat1, lon1, _ = gps_data[i-1]
        t2, lat2, lon2, _ = gps_data[i]

        dist = haversine(lat1, lon1, lat2, lon2)  # meters
        dt = t2 - t1  # seconds
        if dt > 0:
            speed = dist / dt
            speeds.append((t2, speed))
    return speeds


In [13]:
from ultralytics import YOLO

# Load model
model = YOLO("vehicle_detection_custom2/weights/best.pt")

# Run detection + tracking on video
results = model.track(
    source="video1.MP4",
    conf=0.01,
    show=True,
    save=True,
    tracker="bytetrack.yaml"  # built-in tracker
)



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/10030) D:\Project_train\video1.MP4: 384x640 20 HVs, 17 cars, 7 trucks, 69.6ms
video 1/1 (frame 2/10030) D:\Project_train\video1.MP4: 384x640 19 HVs, 15 cars, 8 trucks, 62.2ms
video 1/1 (frame 3/10030) D:\Project_train\video1.MP4: 384x640 20 HVs, 18 cars, 6 trucks, 60.4ms
video 1/1 (frame 4/10030) D:\Project_train\video1.MP4: 384x640 20 HVs, 15 cars, 8 trucks, 58.7ms
video 1/1 (frame 5/10030) D:\Project_train\video1.MP4: 384x640 21 HVs, 

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 6220800 bytes in function 'cv::OutOfMemoryError'


In [10]:
import numpy as np

# Example for one vehicle track:
def compute_vehicle_speed(track_points, fps, meters_per_pixel):
    """
    track_points: list of (frame_no, x_center, y_center)
    fps: frames per second
    meters_per_pixel: calibration constant
    """
    speeds = []
    for i in range(1, len(track_points)):
        _, x1, y1 = track_points[i-1]
        _, x2, y2 = track_points[i]

        pixel_dist = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        real_dist = pixel_dist * meters_per_pixel
        speed_mps = real_dist * fps
        speeds.append(speed_mps * 3.6)  # km/h

    return np.mean(speeds) if speeds else 0


In [11]:
import cv2
import numpy as np
from ultralytics import YOLO

# ======== CONFIG =========
MODEL_PATH = "vehicle_detection_custom2/weights/best.pt"
VIDEO_PATH = "video1.MP4"
OUTPUT_PATH = "output_with_speed.mp4"
METERS_PER_PIXEL = 0.03018  # <-- adjust based on calibration
FPS = 24           # <-- set your actual FPS
# ==========================

# Load YOLO model
model = YOLO(MODEL_PATH)

# Open video
cap = cv2.VideoCapture(VIDEO_PATH)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS) or FPS
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))

# Data structures
tracks = {}  # {id: [(frame_no, x, y)]}
frame_no = 0

# Run tracking
for result in model.track(source=VIDEO_PATH, stream=True, tracker="bytetrack.yaml", conf=0.3):
    frame = result.orig_img.copy()
    frame_no += 1

    if result.boxes.id is not None:
        for box, obj_id in zip(result.boxes.xyxy, result.boxes.id):
            obj_id = int(obj_id.item())
            x1, y1, x2, y2 = map(int, box[:4])
            cx, cy = int((x1 + x2)/2), int((y1 + y2)/2)

            # Store track points
            if obj_id not in tracks:
                tracks[obj_id] = []
            tracks[obj_id].append((frame_no, cx, cy))

            # --- Compute speed ---
            if len(tracks[obj_id]) >= 2:
                f1, x1p, y1p = tracks[obj_id][-2]
                f2, x2p, y2p = tracks[obj_id][-1]
                pixel_dist = np.sqrt((x2p - x1p)**2 + (y2p - y1p)**2)
                real_dist = pixel_dist * METERS_PER_PIXEL
                time_diff = (f2 - f1) / fps
                speed_mps = real_dist / time_diff if time_diff > 0 else 0
                speed_kmph = speed_mps * 3.6
            else:
                speed_kmph = 0

            # --- Draw box ---
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # --- Draw smaller, sharper speed text ---
            text = f"{speed_kmph:.1f} km/h"
            cv2.putText(frame, text, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()
print(f"✅ Video saved at: {OUTPUT_PATH}")



video 1/1 (frame 1/10030) D:\Project_train\video1.MP4: 384x640 (no detections), 82.9ms
video 1/1 (frame 2/10030) D:\Project_train\video1.MP4: 384x640 1 car, 84.3ms
video 1/1 (frame 3/10030) D:\Project_train\video1.MP4: 384x640 1 car, 63.8ms
video 1/1 (frame 4/10030) D:\Project_train\video1.MP4: 384x640 (no detections), 79.0ms
video 1/1 (frame 5/10030) D:\Project_train\video1.MP4: 384x640 1 car, 96.4ms
video 1/1 (frame 6/10030) D:\Project_train\video1.MP4: 384x640 1 car, 132.7ms
video 1/1 (frame 7/10030) D:\Project_train\video1.MP4: 384x640 (no detections), 107.4ms
video 1/1 (frame 8/10030) D:\Project_train\video1.MP4: 384x640 (no detections), 92.7ms
video 1/1 (frame 9/10030) D:\Project_train\video1.MP4: 384x640 1 car, 89.7ms
video 1/1 (frame 10/10030) D:\Project_train\video1.MP4: 384x640 (no detections), 83.5ms
video 1/1 (frame 11/10030) D:\Project_train\video1.MP4: 384x640 (no detections), 62.3ms
video 1/1 (frame 12/10030) D:\Project_train\video1.MP4: 384x640 1 car, 69.0ms
video 1/1 